Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений

In [1]:
import pandas as pd

In [2]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [3]:
def filter_ (param):
    if ('yandex' in param.traffic_source) or ('google' in param.traffic_source):
        return 'organic'
    elif ('email' in param.traffic_source) or ('paid' in param.traffic_source) and ('Russia' in param.region):
        return 'ad'
    elif ('email' in param.traffic_source) or ('paid' in param.traffic_source) and (not 'Russia' in param.region):
        return 'other'
    else:
        return param.traffic_source

log['source_type'] = log.apply(filter_, axis=1)
log.head()
    

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [4]:
import re
data = pd.read_table('URLs.txt')

data[data.url.str.contains(r'\/\d{8}\-')].head(10)

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
10,/middleeast/36131117-divizion-s-400-ne-zametil...
11,/economics/36065674-rossiyane-vozmutilis-minim...
12,/politics/36118047-otets-24-detey-potreboval-i...
13,/travel/36194479-v-sheremetevo-passazhiry-ustr...
14,/politics/35638742-nesolidno-mariya-zaharova-s...


Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [5]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
ratings_top = ratings.pivot_table(index = 'userId', values = 'rating', aggfunc = 'count', fill_value = 0)
ratings_top.reset_index()

,userId,rating
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100
...,...,...
666,667,68
667,668,20
668,669,37
669,670,31


In [20]:
ratings_timestamp = ratings.groupby('userId').agg({'timestamp': ['min', 'max', 'count']})
ratings_timestamp.reset_index()

userId   timestamp                  
                   min         max count
0        1  1260759108  1260759205    20
1        2   835355395   835356246    76
2        3  1298861589  1298932787    51
3        4   949778714   949982274   204
4        5  1163373044  1163375145   100
..     ...         ...         ...   ...
666    667   847271220   847272234    68
667    668   993613196   993613478    20
668    669  1015829081  1015829766    37
669    670   938781328   940944033    31
670    671  1063500751  1074784735   115

[671 rows x 4 columns]

In [14]:
ratings_timestamp_100 = ratings_timestamp.loc[ratings_timestamp.timestamp['count'] > 100]
ratings_timestamp_100.head()

timestamp                  
               min         max count
userId                              
4        949778714   949982274   204
8       1154389340  1154474527   116
15       997937239  1469330735  1700
17      1127468587  1127476640   363
19       855190091   855195373   423

In [19]:
ratings_timestamp_100['LTV'] = ratings_timestamp_100.timestamp['max'] - ratings_timestamp_100.timestamp['min']
ratings_timestamp_100.reset_index()

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


userId   timestamp                          LTV
                   min         max count           
0        4   949778714   949982274   204     203560
1        8  1154389340  1154474527   116      85187
2       15   997937239  1469330735  1700  471393496
3       17  1127468587  1127476640   363       8053
4       19   855190091   855195373   423       5282
..     ...         ...         ...   ...        ...
253    656   986240991   986244044   128       3053
254    659   834598040   866207451   142   31609411
255    664  1343731283  1441911722   519   98180439
256    665   992836298  1046967769   434   54131471
257    671  1063500751  1074784735   115   11283984

[258 rows x 5 columns]

Задание 4
Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).
Необходимо сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [50]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [54]:
#Группировка по ID
rzd_group = rzd.groupby('client_id').sum()
auto_group = auto.groupby('client_id').sum()
air_group = air.groupby('client_id').sum()

In [59]:
#Соединение таблиц с максимальным сохранением данных. Таблица 1 без адресов
grouped_rzd_auto = rzd_group.join(auto_group, how='outer')
grouped_rzd_auto_air = grouped_rzd_auto.join(air_group, how='outer')
grouped_rzd_auto_air.reset_index()

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [58]:
#Таблица 2 с адресами
cl_base_index = client_base.set_index('client_id')
grouped_adres = grouped_rzd_auto_air.join(cl_base_index, how='outer')
grouped_adres.reset_index()

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
